In [1]:
import os
import pandas as pd
import kagglehub
import humanize

from sklearn.model_selection import train_test_split
from IPython.display import display, HTML

/opt/homebrew/Caskroom/miniconda/base/envs/automl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = os.path.join(".", "..", "data")
datasets_dir = os.path.join(data_dir, "datasets")
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

os.makedirs(datasets_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Load Data

In [3]:
raw_datasets_paths = (
    kagglehub.dataset_download(
        "gabrielluizone/us-domestic-flights-delay-prediction-2013-2018"
    ),
    kagglehub.dataset_download("threnjen/2019-airline-delays-and-cancellations"),
    kagglehub.dataset_download("ulrikthygepedersen/airlines-delay"),
    kagglehub.dataset_download("robikscube/flight-delay-dataset-20182022"),
    kagglehub.dataset_download("mikhail1681/airline-quality-ratings"),
)

In [4]:
for dataset_path in raw_datasets_paths:
    print(
        "Dataset " + dataset_path[dataset_path.index("datasets/") + len("datasets/") :]
    )
    print("\t" + "\n\t".join(sorted(os.listdir(dataset_path))))

Dataset gabrielluizone/us-domestic-flights-delay-prediction-2013-2018/versions/3
	csv_flight
	flight_delay_predict.csv
Dataset threnjen/2019-airline-delays-and-cancellations/versions/9
	full_data_flightdelay.csv
	raw_data
	raw_data_documentation.txt
	test.csv
	train.csv
	train_sets_documentation.txt
Dataset ulrikthygepedersen/airlines-delay/versions/1
	airlines_delay.csv
Dataset robikscube/flight-delay-dataset-20182022/versions/4
	Airlines.csv
	Combined_Flights_2018.csv
	Combined_Flights_2018.parquet
	Combined_Flights_2019.csv
	Combined_Flights_2019.parquet
	Combined_Flights_2020.csv
	Combined_Flights_2020.parquet
	Combined_Flights_2021.csv
	Combined_Flights_2021.parquet
	Combined_Flights_2022.csv
	Combined_Flights_2022.parquet
	raw
	readme.html
	readme.md
Dataset mikhail1681/airline-quality-ratings/versions/1
	Airline Quality Ratings.csv


# Dataset creation

In [5]:
datasets_paths = (
    os.path.join(raw_datasets_paths[0], "flight_delay_predict.csv"),
    os.path.join(raw_datasets_paths[1], "full_data_flightdelay.csv"),
    os.path.join(raw_datasets_paths[2], "airlines_delay.csv"),
    os.path.join(raw_datasets_paths[3], "Combined_Flights_2022.csv"),
    os.path.join(raw_datasets_paths[4], "Airline Quality Ratings.csv"),
)

In [6]:
for i, dataset_path in enumerate(datasets_paths):
    df = pd.read_csv(dataset_path)
    original_shape = df.shape
    df = df.head(10**4)
    memory_usage = df.memory_usage(deep=True).sum()
    memory_usage_humanized = humanize.naturalsize(memory_usage)

    print(
        f"Dataset {i} - {df.shape[0]} x {df.shape[1]}. Space usage {memory_usage_humanized}. Original length is {original_shape[0]}."
    )
    df.to_csv(os.path.join(datasets_dir, f"dataset_{i}.csv"), index=False)

Dataset 0 - 10000 x 20. Space usage 4.8 MB. Original length is 1635590.
Dataset 1 - 10000 x 26. Space usage 4.8 MB. Original length is 6489062.
Dataset 2 - 10000 x 8. Space usage 2.2 MB. Original length is 539382.
Dataset 3 - 10000 x 61. Space usage 14.8 MB. Original length is 4078318.
Dataset 4 - 10000 x 24. Space usage 4.8 MB. Original length is 129880.


# Data loading and Analysis

For this task we will use 6 dataframes that include Airlines/Flight information. Each of them has a binary target variable. In the first part we will analyze the data, check column types, missing values, identify targets etc.

In [7]:
datasets = []

for i in range(len(datasets_paths)):
    df = pd.read_csv(os.path.join(datasets_dir, f"dataset_{i}.csv"))

    print("-" * 100)
    print(f"----> Dataset {i}")
    print("-" * 100)
    df.info()
    display(HTML(df.describe().to_html()))
    display(HTML(df.head(5).to_html()))

    datasets.append(df)

----------------------------------------------------------------------------------------------------
----> Dataset 0
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   is_delay           10000 non-null  float64
 1   Year               10000 non-null  int64  
 2   Quarter            10000 non-null  int64  
 3   Month              10000 non-null  int64  
 4   DayofMonth         10000 non-null  int64  
 5   DayOfWeek          10000 non-null  int64  
 6   FlightDate         10000 non-null  object 
 7   Reporting_Airline  10000 non-null  object 
 8   Origin             10000 non-null  object 
 9   OriginState        10000 non-null  object 
 10  Dest               10000 non-null  object 
 11  DestState          10000 non-null  object 
 1

,is_delay,Year,Quarter,Month,DayofMonth,DayOfWeek,CRSDepTime,Cancelled,Diverted,Distance,DistanceGroup,ArrDelay,ArrDelayMinutes,AirTime
count,10000.000000,10000.0,10000.0,10000.0,10000.000000,10000.000000,10000.000000,10000.0,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.336400,2014.0,1.0,1.0,7.601100,3.939500,1313.922500,0.0,0.0,1001.500200,4.392500,17.501600,22.294500,134.300300
std,0.472501,0.0,0.0,0.0,4.142354,1.970946,475.313405,0.0,0.0,514.227925,2.020013,48.150718,45.288576,61.603448
min,0.000000,2014.0,1.0,1.0,1.000000,1.000000,15.000000,0.0,0.0,224.000000,1.000000,-56.000000,0.000000,32.000000
25%,0.000000,2014.0,1.0,1.0,4.000000,2.000000,920.000000,0.0,0.0,641.000000,3.000000,-8.000000,0.000000,89.000000
50%,0.000000,2014.0,1.0,1.0,8.000000,4.000000,1315.000000,0.0,0.0,868.000000,4.000000,3.000000,3.000000,122.000000
75%,1.000000,2014.0,1.0,1.0,11.000000,6.000000,1700.000000,0.0,0.0,1440.000000,6.000000,25.000000,25.000000,179.000000
max,1.000000,2014.0,1.0,1.0,15.000000,7.000000,2359.000000,0.0,0.0,2139.000000,9.000000,1175.000000,1175.000000,320.000000


,is_delay,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Origin,OriginState,Dest,DestState,CRSDepTime,Cancelled,Diverted,Distance,DistanceGroup,ArrDelay,ArrDelayMinutes,AirTime
0,1.0,2014,1,1,1,3,2014-01-01,UA,LAX,CA,ORD,IL,900,0.0,0.0,1744.0,7,43.0,43.0,218.0
1,0.0,2014,1,1,1,3,2014-01-01,AA,IAH,TX,DFW,TX,1750,0.0,0.0,224.0,1,2.0,2.0,50.0
2,1.0,2014,1,1,1,3,2014-01-01,AA,LAX,CA,ORD,IL,1240,0.0,0.0,1744.0,7,26.0,26.0,220.0
3,1.0,2014,1,1,1,3,2014-01-01,AA,DFW,TX,LAX,CA,1905,0.0,0.0,1235.0,5,159.0,159.0,169.0
4,0.0,2014,1,1,1,3,2014-01-01,AA,DFW,TX,CLT,NC,1115,0.0,0.0,936.0,4,-13.0,0.0,108.0


----------------------------------------------------------------------------------------------------
----> Dataset 1
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   MONTH                          10000 non-null  int64  
 1   DAY_OF_WEEK                    10000 non-null  int64  
 2   DEP_DEL15                      10000 non-null  int64  
 3   DEP_TIME_BLK                   10000 non-null  object 
 4   DISTANCE_GROUP                 10000 non-null  int64  
 5   SEGMENT_NUMBER                 10000 non-null  int64  
 6   CONCURRENT_FLIGHTS             10000 non-null  int64  
 7   NUMBER_OF_SEATS                10000 non-null  int64  
 8   CARRIER_NAME                   10000 non-null  object 
 9   AIRPORT_F

,MONTH,DAY_OF_WEEK,DEP_DEL15,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,AIRLINE_AIRPORT_FLIGHTS_MONTH,AVG_MONTHLY_PASS_AIRPORT,AVG_MONTHLY_PASS_AIRLINE,FLT_ATTENDANTS_PER_PASS,GROUND_SERV_PER_PASS,PLANE_AGE,LATITUDE,LONGITUDE,PRCP,SNOW,SNWD,TMAX,AWND
count,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.0,4.528000,0.166700,4.041300,2.938200,31.430900,138.697700,14434.910100,60859.256200,3662.785500,2.029687e+06,8.190901e+06,0.000109,0.000141,11.589400,36.672197,-99.601710,0.038763,0.148230,0.413910,53.996500,7.132131
std,0.0,1.034854,0.372726,2.542015,1.686317,20.880283,46.962224,8388.079846,30598.482486,4395.428037,1.169187e+06,4.779479e+06,0.000086,0.000046,6.996764,5.872931,18.028152,0.105936,0.522305,1.215276,15.310341,3.479158
min,1.0,3.000000,0.000000,1.000000,1.000000,1.000000,44.000000,1282.000000,6713.000000,7.000000,9.061100e+04,4.737940e+05,0.000000,0.000007,1.000000,21.319000,-157.922000,0.000000,0.000000,0.000000,28.000000,2.010000
25%,1.0,3.000000,0.000000,2.000000,2.000000,15.000000,110.000000,8569.000000,24623.000000,989.000000,1.065782e+06,3.190369e+06,0.000034,0.000099,5.000000,33.641000,-117.187000,0.000000,0.000000,0.000000,34.000000,4.250000
50%,1.0,5.000000,0.000000,3.000000,3.000000,27.000000,143.000000,13056.000000,62105.000000,2896.000000,1.903352e+06,8.501631e+06,0.000098,0.000149,12.000000,36.127000,-93.218000,0.000000,0.000000,0.000000,60.000000,6.930000
75%,1.0,5.000000,0.000000,5.000000,4.000000,41.000000,173.000000,23400.000000,75506.000000,4691.000000,3.103410e+06,1.246018e+07,0.000144,0.000177,17.000000,41.978000,-84.427000,0.000000,0.000000,0.000000,65.000000,9.170000
max,1.0,7.000000,1.000000,11.000000,13.000000,93.000000,337.000000,30842.000000,107363.000000,18809.000000,4.365661e+06,1.338300e+07,0.000348,0.000229,32.000000,47.447000,-71.006000,0.620000,4.400000,5.900000,77.000000,18.120000


...

,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginStateFips,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestStateFips,DestWac,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings
count,10000.000000,9154.000000,9154.000000,9154.000000,9116.000000,9104.000000,9104.000000,10000.000000,9104.000000,10000.000000,10000.0,10000.0,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.00000,9154.000000,9154.000000,9135.000000,9135.000000,9116.000000,9116.000000,10000.000000,9104.000000,9104.000000,9104.000000,10000.000000,10000.00000
mean,1325.796900,1350.206358,34.307188,32.944177,1426.690983,31.914433,96.371485,125.083300,117.971441,695.657200,2022.0,2.0,4.0,2.200600,5.626600,19499.346400,2478.791900,19555.726000,2478.460400,12573.125400,1.257316e+06,31598.780500,26.344600,58.265400,12749.426900,1.274946e+06,31691.514800,25.612300,58.38510,0.448547,1.569041,14.523043,1370.388068,1424.213142,7.076349,1468.284200,26.932008,0.404547,1.203976,3.270900,0.00470
std,485.313028,509.643626,56.685967,57.568865,564.068123,56.926456,55.963964,55.980784,58.243264,461.070076,0.0,0.0,0.0,1.916853,1.070461,225.392593,1405.609041,356.185267,1405.276081,1587.991814,1.587991e+05,1241.647913,16.889416,24.463307,1495.492975,1.495493e+05,1237.994662,16.747401,24.62935,0.497373,3.258030,11.203030,510.405311,559.856655,8.554385,523.700835,60.163990,0.490831,3.398247,1.852689,0.13813
min,500.000000,2.000000,0.000000,-20.000000,1.000000,0.000000,16.000000,35.000000,31.000000,67.000000,2022.0,2.0,4.0,1.000000,1.000000,19393.000000,1.000000,19393.000000,1.000000,10135.000000,1.013506e+06,30135.000000,1.000000,2.000000,10135.000000,1.013506e+06,30135.000000,1.000000,2.00000,0.000000,-2.000000,3.000000,1.000000,1.000000,1.000000,5.000000,-49.000000,0.000000,-2.000000,1.000000,0.00000
25%,915.000000,924.000000,0.000000,-1.000000,1032.000000,0.000000,55.000000,85.000000,75.000000,349.000000,2022.0,2.0,4.0,1.000000,5.000000,19393.000000,1228.000000,19393.000000,1228.000000,11193.000000,1.119302e+06,30599.000000,12.000000,35.000000,11292.000000,1.129202e+06,30693.000000,12.000000,36.00000,0.000000,-1.000000,9.000000,937.000000,1029.000000,4.000000,1055.000000,-9.000000,0.000000,-1.000000,2.000000,0.00000
50%,1325.000000,1338.000000,10.000000,10.000000,1443.000000,6.000000,83.000000,115.000000,104.000000,595.000000,2022.0,2.0,4.0,2.000000,6.000000,19393.000000,2527.500000,19393.000000,2527.500000,12339.000000,1.233904e+06,31123.000000,24.000000,64.000000,12892.000000,1.289208e+06,31453.000000,22.000000,64.00000,0.000000,0.000000,12.000000,1352.000000,1441.000000,5.000000,1505.000000,6.000000,0.000000,0.000000,3.000000,0.00000
75%,1730.000000,1753.000000,44.000000,44.000000,1858.000000,40.000000,123.000000,150.000000,145.000000,925.000000,2022.0,2.0,4.0,2.000000,6.000000,19393.000000,3777.000000,19393.000000,3776.250000,13930.000000,1.393007e+06,32467.000000,47.000000,81.000000,13930.000000,1.393007e+06,32467.000000,44.000000,82.00000,1.000000,2.000000,16.000000,1807.500000,1855.000000,7.000000,1905.000000,40.000000,1.000000,2.000000,4.000000,0.00000
max,2255.000000,2358.000000,659.000000,659.000000,2400.000000,697.000000,388.000000,425.000000,474.000000,2979.000000,2022.0,2.0,4.0,15.000000,7.000000,19977.000000,6665.000000,20500.000000,6665.000000,15919.000000,1.591904e+06,35412.000000,72.000000,93.000000,15919.000000,1.591904e+06,35412.000000,72.000000,93.00000,1.000000,12.000000,168.000000,2400.000000,2400.000000,186.000000,2355.0000

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,1228.0,0.0,40.0,72.0,65.0,212.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4301,C5,20445,C5,N21144,4301,11921,1192102,31921,"Grand Junction, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1100-1159,17.0,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,848.0,0.0,55.0,77.0,80.0,295.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4299,C5,20445,C5,N16170,4299,12206,1220605,32206,"Harlingen/San Benito, TX",TX,48,Texas,74,12266,1226603,31453,"Houston, TX",TX,48,Texas,74,0.0,-1.0,0700-0759,16.0,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,1636.0,0.0,47.0,70.0,82.0,251.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4298,C5,20445,C5,N21144,4298,11413,1141307,30285,"Durango, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1500-1559,21.0,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,1547.0,0.0,57.0,90.0,77.0,376.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4296,C5,20445,C5,N11184,4296,12266,1226603,31453,"Houston, TX",TX,48,Texas,74,11973,1197302,31973,"Gulfport/Biloxi, MS",MS,28,Mississippi,53,0.0,-1.0,1400-1459,16.0,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,1251.0,6.0,49.0,70.0,76.0,251.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4295,C5,20445,C5,N17146,4295,11413,1141307,30285,"Durango, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,0.0,1100-1159,19.0,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0


----------------------------------------------------------------------------------------------------
----> Dataset 4
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   ID                                      10000 non-null  int64  
 1   Gender                                  10000 non-null  object 
 2   Age                                     10000 non-null  int64  
 3   Customer Type                           10000 non-null  object 
 4   Type of Travel                          10000 non-null  object 
 5   Class                                   10000 non-null  object 
 6   Flight Distance                         10000 non-null  int64  
 7   Departure Delay                         10000 no

,ID,Age,Flight Distance,Departure Delay,Arrival Delay,Departure and Arrival Time Convenience,Ease of Online Booking,Check-in Service,Online Boarding,Gate Location,On-board Service,Seat Comfort,Leg Room Service,Cleanliness,Food and Drink,In-flight Service,In-flight Wifi Service,In-flight Entertainment,Baggage Handling
count,10000.00000,10000.000000,10000.00000,10000.000000,9962.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,40.365700,1078.28090,15.599900,18.383256,3.048400,2.728000,3.413000,3.35130,2.971800,3.460100,3.526200,3.418200,3.331400,3.202100,3.736800,2.643900,3.371900,3.744200
std,2886.89568,15.127539,1112.49462,43.570727,46.108930,1.542302,1.445346,1.216051,1.35354,1.283888,1.268057,1.300339,1.310145,1.293268,1.308903,1.141077,1.338608,1.340735,1.143283
min,1.00000,7.000000,67.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,2500.75000,28.000000,299.00000,0.000000,0.000000,2.000000,2.000000,3.000000,2.00000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,3.000000
50%,5000.50000,41.000000,562.00000,0.000000,0.000000,3.000000,3.000000,4.000000,4.00000,3.000000,4.000000,4.000000,4.000000,3.000000,3.000000,4.000000,3.000000,4.000000,4.000000
75%,7500.25000,52.000000,1638.00000,12.000000,17.000000,4.000000,4.000000,4.000000,4.00000,4.000000,4.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,5.000000,5.000000
max,10000.00000,85.000000,3997.00000,1017.000000,1011.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay,Departure and Arrival Time Convenience,Ease of Online Booking,Check-in Service,Online Boarding,Gate Location,On-board Service,Seat Comfort,Leg Room Service,Cleanliness,Food and Drink,In-flight Service,In-flight Wifi Service,In-flight Entertainment,Baggage Handling,Satisfaction
0,1,Male,48,First-time,Business,Business,821,2,5.0,3,3,4,3,3,3,5,2,5,5,5,3,5,5,Neutral or Dissatisfied
1,2,Female,35,Returning,Business,Business,821,26,39.0,2,2,3,5,2,5,4,5,5,3,5,2,5,5,Satisfied
2,3,Male,41,Returning,Business,Business,853,0,0.0,4,4,4,5,4,3,5,3,5,5,3,4,3,3,Satisfied
3,4,Male,50,Returning,Business,Business,1905,0,0.0,2,2,3,4,2,5,5,5,4,4,5,2,5,5,Satisfied
4,5,Female,49,Returning,Business,Business,3470,0,1.0,3,3,3,5,3,3,4,4,5,4,3,3,3,3,Satisfied


# Identifying targets

In [8]:
datasets[0].rename(columns={"is_delay": "target"}, inplace=True)
datasets[1].rename(columns={"DEP_DEL15": "target"}, inplace=True)
datasets[2].rename(columns={"Class": "target"}, inplace=True)
datasets[3].rename(columns={"DepDel15": "target"}, inplace=True)
datasets[4].rename(columns={"Satisfaction": "target"}, inplace=True)

In [9]:
for i, dataset in enumerate(datasets):
    print(f"Dataset {i}")
    print(
        "\t"
        + "\n\t".join(
            [f"{k}\t{v}" for k, v in dataset["target"].value_counts().to_dict().items()]
        )
    )

Dataset 0
	0.0	6636
	1.0	3364
Dataset 1
	0	8333
	1	1667
Dataset 2
	0	10000
Dataset 3
	0.0	5048
	1.0	4106
Dataset 4
	Neutral or Dissatisfied	5689
	Satisfied	4311


In [10]:
datasets[0]["target"] = datasets[0]["target"].astype(bool)
datasets[1]["target"] = datasets[0]["target"].astype(bool)
datasets[2]["target"] = datasets[0]["target"].astype(bool)
datasets[3]["target"] = datasets[0]["target"].astype(bool)
datasets[4]["target"] = datasets[4]["target"].apply(lambda x: x == "Satisfied")

## Train test split

In [11]:
for i, dataset in enumerate(datasets):
    X = dataset.drop(columns=["target"])
    y = dataset["target"]

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

    X_train_file_path = os.path.join(train_dir, f"X{i}_train.csv")
    y_train_file_path = os.path.join(train_dir, f"y{i}_train.csv")
    X_test_file_path = os.path.join(test_dir, f"X{i}_test.csv")
    y_test_file_path = os.path.join(test_dir, f"y{i}_test.csv")

    X_train.to_csv(X_train_file_path, index=False)
    Y_train.to_csv(y_train_file_path, index=False)
    X_test.to_csv(X_test_file_path, index=False)
    Y_test.to_csv(y_test_file_path, index=False)